## Transforming Data for Mongo DB

This notebook loads csv files of data previously cleaned and loads them to Mongo DB and transforms them to JSON.

* cities - information on the income and poverty levels by city with associated lat and long

---


In [9]:
import pandas as pd 
import os
import numpy as np

import pymongo
import json

import requests


# function to save dataframe to collection_name in MongoDB 'wines'    
def saveMongo(df, collection_name, replace=False):
    mng_client = pymongo.MongoClient('localhost', 27017)
    mng_db = mng_client['guns'] 
    if replace:
        mng_db[collection_name].drop() 
    db_cm = mng_db[collection_name]
    data = df
    data_json = json.loads(data.to_json(orient='records', date_unit='ns'))
    #db_cm.delete_many()
    db_cm.insert_many(data_json)
    


## Load csv files into pandas dataframes, clean, save to mongo db

In [14]:
# read in main wine scrape csv, 2019 US wines from Wine Magazine web site, save to collection "winesall"
cities_path = os.path.join("..","Data","Cities.csv")

df_cities = pd.read_csv(cities_path, encoding="UTF-8")
df_cities.head()

# # save to/replace collection "cities" in "guns" mongo db
# saveMongo(df_cities, "cities", replace=True)


,State,City,percent_completed_hs,poverty_rate,Median Income,Lat,Lng,cityarea,city_name,city_type,county
0,AL,Abanda CDP (Test Country),21.2,78.8,11207,33.098839,-85.527824,Abanda CDP,Abanda,CDP,Test
1,AL,Abbeville city,69.1,29.1,25615,31.571835,-85.250489,Abbeville city,Abbeville,city,NaN
2,AL,Adamsville city,78.9,25.5,42575,33.600939,-86.956106,Adamsville city,Adamsville,city,NaN
3,AL,Addison town,81.4,30.7,37083,34.202324,-87.181401,Addison town,Addison,town,NaN
4,AL,Akron town,68.6,42,21667,32.876517,-87.742510,Akron town,Akron,town,NaN


In [13]:
# placeholder to read in other csvs
guns_path = os.path.join("..","rawdata","gun-violence-data_01-2013_12-2015.csv")

df_guns = pd.read_csv(guns_path, encoding="UTF-8")

df_guns ["n_involved"]= df_guns["n_killed"]+df_guns["n_injured"]

conditions = [
    (df_guns["n_involved"] >= 4),
    (df_guns["n_killed"] == 0)]
choices = ["mass shooting", "injuries only"]
df_guns["type"] = np.select(conditions, choices, default="less than 4 involved")

# df_guns ["type"] = np.where(df_guns["n_involved"]>=4, "mass shooting", "under 4 involved")
df_guns.head()

# save to/replace collection "cities" in "guns" mongo db
saveMongo(df_guns, "guns", replace=True)
